File: prices.ipynb\
Author: alexkobz\
Date: 08-10-2024\
Task: DSRFU-298

#### Шаг 0. Импорт и получение токена

In [1]:
import asyncio
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt, date, timedelta
from functions.divide_chunks import divide_chunks
from math import ceil
from functions.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = last_day_month
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)

In [3]:
from rudata.RuDataDF import RuDataDF
from rudata.RuDataRequest import RuDataRequest
RuDataRequest.set_headers()

Getting token started
Getting token finished


In [139]:
month_year = REPORT_DATE.strftime("%m.%Y")
isin_file = f"ISIN_{month_year}.xlsx"
Prices = pd.read_excel(f"./data/Input/{isin_file}", names=["ISIN"]).dropna().set_index("ISIN")
Prices["Date"] = REPORT_DATE
Prices[["Name", "Type", "Principal", "Currency", "Clean price pct", "Clean price", "Interest", "Price", 
        "Source", "ISIN in DB", "FX Rate", "RUB Price", "CBR discount 6D", "Discount NCC", "NCC collateral"]] = np.nan

#### Шаг 1. RuData

##### Шаг 1.0. RuData stage

In [5]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

FintoolReferenceData started
FintoolReferenceData finished. FintoolReferenceData shape (101882, 186)


,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn
0,3.0,3.0,NaN,NaN,NaN,RU0006766433,3.0,3.0,0.0,3.0,3.0,NaN,NaN,"Акции обыкновенные АО ""РУСАЛ Новокузнецк""",None,None,001,NaN,0.0,None,0.0,NaN,0.10,RUB,5.399717e+07,1-01-10239-F,2003-12-08T00:00:00,РО ФКЦБ России в Сибирском федеральном округе,2004-01-15T00:00:00,NaN,2004-01-15T00:00:00,None,None,5.399717e+07,NaN,None,RNOV,NaN,5.399717e+07,"АО ""РУСАЛ Новокузнецк""",None,None,None,CORPREG,NaN,None,NaN,NaN,НД,2011-11-19T00:00:00,5938000504,2011-11-19T00:00:00,NaN,None,None,None,None,None,None,None,None,None,None,None,Бездокументарная именная,uncertificated registered,Акции прочих резидентов (обыкновенные),Размещен,Placed,Объединение выпусков,обыкновенные,ordinary,None,None,None,NaN,NaN,NaN,NaN,2.0,2.0,11.0,1.0,None,None,None,S112,"JSC ""RUSAL Novokuznetsk"" ORD SHS",None,4383,None,None,"""РУСАЛ Новокузнецкий Алюминиевый Завод"", АО, а...",None,RU0006766433,РУСАЛ Новокузнецк-1-ао,NaN,1-01-10239-F,2003-12-08T00:00:00,ФКЦБ,NaN,None,None,ESVXFR,None,Акция,None,None,NaN,0.0,NaN,None,None,NaN,RU,NaN,RUB,0.10,NaN,NaN,NaN,В обращении,5.399717e+06,5.399717e+07,5.399717e+06,5.399717e+07,None,None,None,NaN,None,None,None,РУСАЛ Новокузнецк,05785276,Металлургический,6979.0,4221000535,RU,Кемеровская область,None,NaN,2004-01-15T00:00:00,None,NaN,2004-01-15T00:00:00,None,0.0,None,Обыкн-ая,Осн-ой,None,2024-11-19T16:50:27,NaN,На обслуживании,NaN,None,NaN,Исключен с РТС 07.02.2001 г.,NaN,None,None,Non

In [6]:
AccruedInterestOnDate = RuDataDF("AccruedInterestOnDate").df
AccruedInterestOnDate.head()

AccruedInterestOnDate started
AccruedInterestOnDate finished. AccruedInterestOnDate shape (101882, 14)


,fintoolId,shiftDays,shiftTypeSource,typeCurrency,date,accruedInterest,accruedInterestPct,currentFaceValue,currentFaceValuePct,calculationType,error,faceValueCurrency,accruedInterestCurrency,faceValueLastKnownDate
0,5019,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '5019': Нет...,None,None,None
1,5014,0,Calendar,None,2024-11-30T00:00:00,NaN,NaN,100.0,100.0,Typical,None,RUB,RUB,2024-11-30T00:00:00
2,4998,0,Calendar,None,2024-11-30T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2024-11-30T00:00:00
3,4995,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '4995': Нет...,None,None,None
4,5034,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '5034': Нет...,None,None,None


In [7]:
RUPriceHistory = RuDataDF("RUPriceHistory").df
RUPriceHistory.shape

RUPriceHistory started
RUPriceHistory finished. RUPriceHistory shape (40536, 82)


(40536, 82)

In [8]:
ExchangeTree = RuDataDF("ExchangeTree").df
ExchangeTree.head()

ExchangeTree started
ExchangeTree finished. ExchangeTree shape (239, 10)


,id,id_parent,shortname_rus,shortname_eng,fullname_rus,fullname_eng,sort_order,update_date,counter,rn
0,144,63,Нал,Cash,Наличная валюта,Cash,0,2020-12-28T17:03:09,239,1
1,270,63,RUDIP 2,RUDIP 2,RUDIP 2,RUDIP 2,1,2022-10-31T19:41:24,239,2
2,84,83,США,US,Макроиндикаторы США,USA macroindicators,1,2020-12-28T17:03:09,239,3
3,85,83,РФ,RU,Макроиндикаторы Россия,Russia macroindicators,2,2020-12-28T17:03:09,239,4
4,2,1,РТС,RTS,РТС индексы,RTS indexes,2,2020-12-28T17:03:09,239,5


In [9]:
MoexSecurities = RuDataDF("MoexSecurities").df
MoexSecurities.head()

MoexSecurities started
MoexSecurities finished. MoexSecurities shape (205203, 24)


,id_iss,secid,boardid,board_title,board_group,board_group_title,market,engine,short_name,isin,cfi_ifx,cfi_name,lot_size,expiration_date,currency,emitent_code,emitent_name,market_sector,min_price_increment,fintoolid,reg_code,update_date,counter,rn
0,2501,EUR_RUB__TOM,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Евро/Рубль-Том,None,YCRSXX,Валютные пары / Тип пары / Кросс-курс,1000,None,RUB,None,None,None,0.0025,NaN,None,2021-02-28T23:56:16.686054,205203,1
1,2502,EUR_RUB__TOD,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Евро/Рубль-Тод,None,YCRSXX,Валютные пары / Тип пары / Кросс-курс,1000,None,RUB,None,None,None,0.0025,NaN,None,2021-02-28T23:56:16.686053,205203,2
2,2840,KZT000000TOM,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Тенге/Рубль-Тод,None,YCRXXX,Валютные пары / Тип пары / Кросс-курс,10000,None,RUB,None,None,None,0.0025,NaN,None,2023-03-23T18:16:31.385678,205203,3
3,3691,UAH000000TOM,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Гривна/Рубль-Тод,None,YCRSXX,Валютные пары / Тип пары / Кросс-курс,1000,None,RUB,None,None,None,0.0025,NaN,None,2018-09-17T09:17:51.410715,205203,4
4,3752,USD000000TOD,CETS,Системные сделки - безадрес.,currency,Системные сделки - безадрес.,selt,currency,Доллар/Рубль-Тод,None,YCRSXX,Валютные пары / Тип пары / Кросс-курс,1000,None,RUB,None,None,None,0.0025,NaN,None,2021-02-28T23:56:17.686097,205203,5


In [10]:
%%time
import aiohttp
import socket
from typing import List, Union, Optional, Dict, Any
from rudata import DocsAPI

isins = Prices[Prices["Price"].isna()].index.unique().tolist()
key = "EndOfDay"
url: str = getattr(DocsAPI, key).url
payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
payloads = []
for isin in isins:
    payload["isin"] = isin
    payloads.append(payload.copy())
    
async def end_of_day_rudata():
    end_of_day = []
    async with aiohttp.ClientSession(
            connector=aiohttp.TCPConnector(limit=DocsAPI.LIMIT,
                                           family=socket.AF_INET),
            trust_env=True,
            timeout=aiohttp.ClientTimeout(3600)
    ) as session:
        for i, chunk in enumerate(divide_chunks(payloads, DocsAPI.LIMIT)):
            tasks: List[asyncio.Task] = [asyncio.create_task(
                RuDataRequest(url, session).post(payload=payload)
            ) for payload in chunk]
            resAll: List[List[dict]] = [await task for task in asyncio.as_completed(tasks, timeout=600)]
            result: List[dict] = [row for task_res in resAll for row in task_res]
            end_of_day.extend(result)
    return end_of_day
    
end_of_day = asyncio.run(end_of_day_rudata())
EndOfDay = pd.DataFrame(end_of_day)
EndOfDay.head()

CPU times: user 7.22 s, sys: 524 ms, total: 7.74 s
Wall time: 27min 54s


,accruedint,admquote,akm_bl_n,akm_e_n,ask,ask_size,ask_size_total,avge_chng,avge_chng_pct,avge_prce,avge_yield,bid,bid_size,bid_size_total,boardid,boardname,buybackdate,buybackprice,cbr_lombard,cbr_pledge,chng,chng_m,chng_m_pct,chng_pct,chng_w,chng_w_pct,chng_y,chng_y_pct,close,consensus_num_est,consensus_time,convexity,convexity_agg,convexity_o,cpn,cpn_date,cpn_period,cpprice,deal_acc,discount,dnt06beg,dnt06max,dnt06min,dnt120beg,dnt120max,dnt120min,dnt14beg,dnt14max,dnt14min,dnt200beg,dnt200max,dnt200min,dnt365beg,dnt365max,dnt365min,duration,duration_agg,duration_n,duration_o,duration_on,exch,exp_bl_n,exp_e_n,facevalue,facevalue_currency,fact_frcst_chng,fch_b_i,fch_b_n,fch_e_dlti,fch_e_lti,forecast,frcst_chng,high,ifx_b_i,ifx_b_n,ifx_e_i,ifx_e_n,isin,issuer,issuesize,last,last_time,last_yield,last2avge,lastsettlecode,lclose,lnkcurrency,lotsize,low,mat_date,mat_date_dt,mcap,mds_b_i,mds_b_n,mds_e_lti,mds_e_ltn,mprice,mprice2,name,open,...,pvbp_o,rec_buy_pct,rec_change,rec_hold_pct,rec_sell_pct,rec_val,recommendation,rus_b_n,rus_e_i,rus_e_n,seccode,secname,secname_e,sector,settlecode,shortname_eng,shortname_rus,snp_b_i,snp_b_n,snp_e_lti,snp_e_n,spread,status,target_price,tick_acc,time,tp_chng,tp_chng_m,tp_chng_m_pct,tp_chng_pct,tp_chng_q,tp_chng_q_pct,tp_chng_time,tp_chng_y,tp_chng_y_pct,tp_deviation,tp_high,tp_low,tp_median,val,val_acc,val_acc_usd,val_usd,vol,vol_acc,vol_lots,y2o_ask,y2o_bid,y2o_change,y2o_change_pct,y2o_last,y2o_prev,y2o_wap,yield_2,yield_agg,yield_ask,yield_average_12m,yield_average_1m,yield_average_2m,yield_average_2w,yield_average_3m,yield_average_6m,yield_average_o12m,yield_average_o1m,yield_average_o2m,yield_average_o2w,yield_average_o3m,yield_average_o6m,yield_bid,yield_chng,yield_cpprice,yield_cwa,yield_high,yield_low,yield_o_cwa,yield_o_swa,yield_open,yield_prev,yield_swa,total_return,full_price_pct,zspread,gspread,currency,fintoolId,id_iss,lcurrentprice,prevlclose,lcp_chng,lcp_chng_pct,bid_price_change,ask_price_change,bid_yield_change,ask_yield_change,id_trade_site,ifrs_level,rd_exch,rd_score,add_date,update_date
0,24.1600,None,None,None,100.4900,None,84150.0,NaN,NaN,NaN,NaN,99.9300,None,110.0,TQCB,Т+: Корп.облигации - безадрес.,2025-04-18T03:00:00,100.0,NaN,None,0.0000,-0.1400,-0.13,0.00,-0.6800,-0.67,1.4700,1.49,99.92,None,None,0.997453,0.339056,0.339056,52.36,2025-01-17T00:00:00,91.0,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,296.0,135.0,0.659835,135.0,0.302078,MICEX,None,None,1000.00,None,None,None,None,None,None,None,None,NaN,None,None,None,None,RU000A0JVWB3,None,10000000.0,99.9200,2024-11-29T18:15:13,NaN,0.0000,None,None,RUB,1.0,NaN,17.10.2025,2025-10-17T00:00:00,NaN,None,None,None,None,100.5500,None,РСХБ БО9.T.MICEX,NaN,...,-0.030912,None,None,None,None,None,None,None,None,None,RU000A0JVWB3,"""Россельхозбанк"" (АО) БО-09",Rosselkhozbank BO-09,None,None,None,None,None,None,None,None,0.5600,None,NaN,None,2024-11-29T00:00:00,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.474023,23.095957,NaN,NaN,NaN,NaN,NaN,None,NaN,21.220275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.889874,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,102.33600,None,None,RUB,100282,1112135,None,None,None,None,None,None,None,None,170,NaN,None,None,2024-11-29T23:55:28,2024-11-29T23:55:28
1,NaN,None,None,None,1344.4000,None,0.0,31.9,2.42,1346.80,NaN,1344.0000,None,0.0,TQBR,T+ Акции и ДР,None,NaN,NaN,None,5.4000,60.8000,4.74,0.40,-110.6000,-7.61,771.0000,134.93,1337.00,None,None,NaN,NaN,NaN,NaN,None,NaN,None,11266.0,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,MICEX,None,None,0.67,None,None,None,None,None,None,None,None,1361.0,None,None,None,None,RU000A0JVW89,None,22977000.0,1342.4000,2024-11-29T23:49:58.0006,NaN,27.5000,None,None,RUB,1.0,1326.8000,None,None,3.094542e+10,None,None,None,None,1346.8000,None,SFIN.T.MICEX,1341.0000,...,NaN,None,None,None,None,None,Покупать,None,No

In [11]:
%%time
import aiohttp
import socket
from typing import List, Union, Optional, Dict, Any
from rudata import DocsAPI
from time import sleep

currencies = pd.read_csv("./data/Input/currencies.csv", header=0, encoding='cp1251', sep=';', usecols=["Currency"])["Currency"].tolist()
key = "CurrencyRate"
url: str = getattr(DocsAPI, key).url
payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
currencies_rudata = []
for currency in currencies:
    payload["from"] = currency
    ans = requests.post(url, json={'from': currency, 'to': 'RUB', 'date': REPORT_DATE_STR}, headers=RuDataRequest.headers)
    result: dict = ans.json()
    result["currency"] = currency
    currencies_rudata.append(result)
    sleep(1)
CurrencyRate = pd.DataFrame(currencies_rudata)
CurrencyRate.head()

CPU times: user 12.3 s, sys: 122 ms, total: 12.4 s
Wall time: 3min 35s


,rate,endOfDayRate,error,currency
0,29.337200,29.337200,None,AED
1,NaN,NaN,None,AFN
2,NaN,NaN,None,ALL
3,NaN,NaN,None,KMF
4,0.273011,0.273011,None,AMD


##### Шаг 1.1. Discounts

###### Шаг 1.1.1. CBR

In [140]:
from sources.CBR_Soap import CBR_Soap

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36', 
           'Accept-Encoding': 'gzip, deflate', 
           'Accept': '*/*', 
           'Connection': 'keep-alive'}
cbr_disc = CBR_Soap()\
.get_discounts(date=last_work_date_month_str)[['ISIN', 'от 1 месяца до 180 дней_Beg']]\
.rename(columns={'от 1 месяца до 180 дней_Beg': 'CBR discount 6D'})\
.set_index('ISIN')

GET http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?wsdl using httplib2 0.20.2
GET http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?wsdl using httplib2 0.20.2
GET http://www.cbr.ru/secinfo/secinfo.asmx?wsdl using httplib2 0.20.2
GET http://www.cbr.ru/secinfo/secinfo.asmx?wsdl using httplib2 0.20.2


/home/alexander/PycharmProjects/insurance/sources/CBR_Soap.py:212: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  disc = pd.concat([disc, pd.DataFrame(index=[0], data=d)], axis=0)


In [141]:
Prices.update(cbr_disc)
Prices['CBR discount 6D'] = Prices['CBR discount 6D'].fillna(1)

###### Шаг 1.1.2. nationalclearingcentre Рыночные риски

In [142]:
currMarketRatesResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/fondMarketRates?action=xls&xls=1&lang=ru&date={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
currMarketRates = pd.read_excel(currMarketRatesResponse.content,header=2)
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)].reset_index(drop=True)
currMarketRates = currMarketRates\
.merge(MoexSecurities[['secid', 'isin']], how='left', left_on='Инструмент', right_on='secid')\
.rename(columns={'isin': 'ISIN'})\
.groupby('ISIN')\
.last()
currMarketRates = currMarketRates[currMarketRates.index.str.match('([A-Z]{2})([A-Z0-9]{9})([0-9]{1})$')]
currMarketRates = currMarketRates[(currMarketRates['Диапазон']==1)&(currMarketRates['Ставка риска падения цены,%']!=100)]
currMarketRates=currMarketRates[['Ставка риска падения цены,%']]\
.rename(columns={'Ставка риска падения цены,%': 'Risk rate'})
currMarketRates['Discount NCC'] = currMarketRates['Risk rate'].apply(lambda x: min(30, ceil(x/np.sqrt(2)))) #Формула из п.12.1 Методики НКЦ определения риск-параметров
currMarketRates /= 100

/tmp/ipykernel_3792/88371159.py:2: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  currMarketRates = pd.read_excel(currMarketRatesResponse.content,header=2)


In [143]:
Prices.update(currMarketRates)
Prices['Discount NCC'] = Prices['Discount NCC'].fillna(1)

###### Шаг 1.1.3. nationalclearingcentre Параметры ценных бумаг

In [144]:
securInfoResponse = requests.get(f'https://www.nationalclearingcentre.ru/api/v1/rates/securInfo?action=xls&lang=ru&settleDate={last_work_date_month.strftime("%d.%m.%Y")}', headers=headers)
securInfo = pd.read_excel(securInfoResponse.content, header=2)[['ISIN ЦБ', 'Принимается в обеспеч.']]\
.rename(columns={
    'ISIN ЦБ': 'ISIN',
    'Принимается в обеспеч.': 'NCC collateral'
})\
.dropna()\
.set_index('ISIN')
securInfo = securInfo[securInfo['NCC collateral']=='Да']

/tmp/ipykernel_3792/1680612647.py:2: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  securInfo = pd.read_excel(securInfoResponse.content, header=2)[['ISIN ЦБ', 'Принимается в обеспеч.']]\


In [145]:
Prices.update(securInfo)
Prices['NCC collateral'] = Prices['NCC collateral'].fillna('Нет')

/tmp/ipykernel_3792/3081263276.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan 'Да' nan ... nan 'Да' nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Prices.update(securInfo)


##### Шаг 1.2. RuData transform

In [146]:
TransformFintoolReferenceData = FintoolReferenceData[['isincode', 'nickname', 'fintooltype', 'fintoolid', 'faceftname']]\
.rename(columns={
    "isincode": "ISIN",
    "nickname": "Name",
    "fintoolid": "Fintool",
    "fintooltype": "Type", 
    "faceftname": "Currency"
})
TransformFintoolReferenceData["ISIN in DB"] = 1
TransformFintoolReferenceData["Source"] = 'RU Data (ЦЦ НРД)'
TransformFintoolReferenceData = TransformFintoolReferenceData.merge(
    AccruedInterestOnDate[['fintoolId', 'accruedInterest', 'currentFaceValue']], how="left", left_on="Fintool", right_on="fintoolId")
TransformFintoolReferenceData = TransformFintoolReferenceData\
.rename(columns={
    'accruedInterest': 'Interest', 
    'currentFaceValue': 'Principal'})\
.drop(columns=["fintoolId"])

In [147]:
RUPriceHistoryLast = RUPriceHistory.groupby('isincode').last().rename(columns={
    'vp_pct': 'Clean price pct', 
    'vp_pc': 'Clean price',
    'acc_int': 'InterestHistory'
}).reset_index()

In [148]:
TransformFintoolReferenceData = TransformFintoolReferenceData.merge(
    RUPriceHistoryLast[['isincode', 'Clean price pct', 'Clean price', 'InterestHistory']], how='left', left_on="ISIN", right_on="isincode"
).drop(columns=["isincode"])
TransformFintoolReferenceData["Interest"] = TransformFintoolReferenceData["Interest"].combine_first(TransformFintoolReferenceData["InterestHistory"])
TransformFintoolReferenceData["Interest"].fillna(0, inplace=True)
TransformFintoolReferenceData.drop(columns="InterestHistory", inplace=True)

/tmp/ipykernel_3792/539574806.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  TransformFintoolReferenceData["Interest"].fillna(0, inplace=True)


In [149]:
bonds_index = TransformFintoolReferenceData[TransformFintoolReferenceData["Type"]=='Облигация'].index

In [150]:
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Price"] = \
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Clean price"] + \
TransformFintoolReferenceData.loc[TransformFintoolReferenceData.index.isin(bonds_index), "Interest"]

TransformFintoolReferenceData.loc[~TransformFintoolReferenceData.index.isin(bonds_index), "Price"] = \
TransformFintoolReferenceData.loc[~TransformFintoolReferenceData.index.isin(bonds_index), "Clean price"]

In [151]:
Prices.update(TransformFintoolReferenceData.groupby("ISIN").first())

/tmp/ipykernel_3792/3210158000.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Банк ВТБ-С-1-1135' 'РСХБ-9-боб' 'EFG Internat-14-2026-евро' ...
 'Томская Обл-34062-об' 'РЖД-001P-04R' 'РуссНефть НК-1-ао']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Prices.update(TransformFintoolReferenceData.groupby("ISIN").first())
/tmp/ipykernel_3792/3210158000.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Облигация' 'Облигация' 'Облигация' ... 'Облигация' 'Облигация' 'Акция']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Prices.update(TransformFintoolReferenceData.groupby("ISIN").first())
/tmp/ipykernel_3792/3210158000.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['RUB' 'RUB' 

In [152]:
exch_prices = EndOfDay.merge(
    ExchangeTree[['id', 'fullname_rus']], how='left', left_on='id_trade_site', right_on='id')[['fintoolId', 'last', 'fullname_rus']]
exch_prices["Source"] = exch_prices['fullname_rus'].apply(lambda x: 'RU Data (' + str(x) + ')')
exch_prices = exch_prices\
.rename(columns={
    'fintoolId': 'Fintool',
    'last': 'PriceEX'
})\
.drop(columns="fullname_rus")
exch_prices = exch_prices.merge(TransformFintoolReferenceData[["Fintool", "ISIN", "Type", "Principal", "Interest"]], how='inner')

In [153]:
exch_bonds = exch_prices[exch_prices['Type']=='Облигация']
exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
exch_bonds["Price"] = exch_bonds["Clean price"] + exch_bonds["Interest"]

/tmp/ipykernel_3792/1720937637.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_bonds["Clean price pct"] = exch_bonds["PriceEX"]
/tmp/ipykernel_3792/1720937637.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_bonds["Clean price"] = exch_bonds["Clean price pct"]/100 * exch_bonds["Principal"]
/tmp/ipykernel_3792/1720937637.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [154]:
exch_not_bonds = exch_prices[exch_prices['Type']!='Облигация']
exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
exch_not_bonds['Price'] = exch_not_bonds['PriceEX']

/tmp/ipykernel_3792/3772375966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_not_bonds['Clean price'] = exch_not_bonds['PriceEX']
/tmp/ipykernel_3792/3772375966.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exch_not_bonds['Price'] = exch_not_bonds['PriceEX']


In [155]:
Prices.update(exch_bonds.set_index("ISIN"))
Prices.update(exch_not_bonds.set_index("ISIN"))

In [113]:
ResultPrices = Prices[~Prices['Price'].isna()]
Prices = Prices[Prices['Price'].isna()]

In [156]:
ResultPrices.shape

(1141, 16)

In [157]:
Prices.shape

(3259, 16)

#### Шаг 2. Cbonds

##### Шаг 2.1. Bonds

In [158]:
tradings_bonds = pd.read_excel(f'./data/Input/tradings_bonds.xls')
tradings_bonds_1 = pd.read_excel(f'./data/Input/tradings_bonds_1.xls')

tradings_bonds = pd.concat([tradings_bonds, tradings_bonds_1], axis=0)
tradings_bonds = tradings_bonds[['ISIN', 'Краткое название эмитента (rus)', 'Валюта', 'Индикативная цена', 'Номинал по еврооблигациям']]\
.rename(columns={
    'Краткое название эмитента (rus)': 'Name', 
    'Валюта': 'Currency', 
    'Индикативная цена': 'Clean price pct', 
    'Номинал по еврооблигациям': 'Principal EB'
})

/tmp/ipykernel_3792/1211491189.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tradings_bonds = pd.concat([tradings_bonds, tradings_bonds_1], axis=0)


In [159]:
tradings_bonds = tradings_bonds.merge(Prices[["Interest", "Principal"]], how='inner', left_on="ISIN", right_index=True)
principal_idx = tradings_bonds[tradings_bonds['Principal']>tradings_bonds['Principal EB']].index
tradings_bonds.loc[principal_idx, 'Principal'] /= tradings_bonds.loc[principal_idx, 'Principal EB']
tradings_bonds.loc[principal_idx, 'Interest'] /= tradings_bonds.loc[principal_idx, 'Principal']

In [160]:
Prices.update(tradings_bonds.set_index("ISIN"))

In [161]:
tradings_nkd = pd.read_excel(f'./data/Input/tradings_nkd.xls')
tradings_nkd_1 = pd.read_excel(f'./data/Input/tradings_nkd_1.xls')
tradings_nkd_2 = pd.read_excel(f'./data/Input/tradings_nkd_2.xls')
tradings_nkd_3 = pd.read_excel(f'./data/Input/tradings_nkd_3.xls')

tradings_nkd = pd.concat([tradings_nkd, tradings_nkd_1, tradings_nkd_2, tradings_nkd_3], axis=0)
tradings_nkd = tradings_nkd[['ISIN код эмиссии', 'Накопленный купонный доход', 'Непогашенный номинал']]\
.rename(columns={
    'ISIN код эмиссии': "ISIN", 
    'Накопленный купонный доход': "Interest", 
    'Непогашенный номинал': "Principal"
})
tradings_nkd = tradings_nkd.groupby("ISIN").first()

In [162]:
Prices.update(tradings_nkd)

In [163]:
tradings_new = pd.read_excel(f'./data/Input/tradings_new_{REPORT_DATE_STR}.xls')

tradings_new = tradings_new[['Код ISIN', 'Эмиссия', 'Дата', 'Bid, закр.', 'НКД']]\
.rename(columns={
    'Код ISIN': 'ISIN',
    'Эмиссия': 'Name',
    'Дата': 'Date',
    'Bid, закр.': 'Clean price pct',
    'НКД': 'Interest'
})
tradings_new["Max date"] = tradings_new.groupby('ISIN')["Date"].transform('max')
tradings_new = tradings_new.groupby('ISIN').first()

In [164]:
emissions = pd.read_excel(f'./data/Input/emissions_{REPORT_DATE_STR}.xls')

emissions = emissions[['Валюта', 'код ISIN', 'Непогашенный номинал']]\
.rename(columns={
    'код ISIN': 'ISIN',
    'Валюта': 'Currency',
    'Непогашенный номинал': 'Principal'
})
emissions = emissions[~((emissions["ISIN"].isna()) | (emissions["Principal"].isna()) | (emissions["Principal"]==0))].set_index("ISIN")
tradings_new = tradings_new.merge(emissions, how="left", left_index=True, right_index=True)

In [165]:
Prices.update(tradings_new)

In [166]:
Prices.loc[Prices['Interest'] > 0.2*Prices['Principal'], 'Interest'] = 0
Prices['Clean price'] = Prices['Clean price pct']/100 * Prices['Principal']
Prices['Price'] = Prices['Clean price'] + Prices['Interest']

In [167]:
cbonds_bonds_idx = Prices[~Prices["Price"].isna()].index
Prices.loc[cbonds_bonds_idx, "Type"] = 'Облигация'
Prices.loc[cbonds_bonds_idx, 'Source'] = 'Cbonds'
Prices.loc[cbonds_bonds_idx, 'ISIN in DB'] = 1

In [168]:
ResultPrices = pd.concat([ResultPrices, Prices.loc[cbonds_bonds_idx]], axis=0)
Prices = Prices[Prices['Price'].isna()]

In [169]:
Prices.shape

(2222, 16)

In [170]:
ResultPrices.shape

(2178, 16)

##### Шаг 2.2. Stocks

In [171]:
tradings_stocks_all = pd.read_excel(f'./data/Input/tradings_stocks_all.xls')

tradings_stocks_all = tradings_stocks_all[['ISIN', 'Краткое название эмитента (rus)', 'Валюта номинала', 'Валюта торгов', 'Цена закрытия']]\
.rename(columns={
    'Краткое название эмитента (rus)': 'Name', 
    'Валюта номинала': 'Currency', 
    'Валюта торгов': 'Trade currency', 
    'Цена закрытия': 'Price'
})

In [172]:
tradings_stocks_all = tradings_stocks_all.merge(CurrencyRate[["currency", "rate"]], how='left', left_on="Trade currency", right_on="currency")\
.rename(columns={'rate': 'trade rate'})
tradings_stocks_all = tradings_stocks_all.merge(CurrencyRate[["currency", "rate"]], how='left', left_on="Currency", right_on="currency")
tradings_stocks_all['RUB Price'] = tradings_stocks_all['Price'] * tradings_stocks_all['trade rate']
tradings_stocks_all['RUB Price'] = tradings_stocks_all.groupby("ISIN")['RUB Price'].transform('mean')
tradings_stocks_all['Currency'] = tradings_stocks_all['Currency'].combine_first(tradings_stocks_all['Trade currency'])
tradings_stocks_all['Clean price'] = tradings_stocks_all['RUB Price'] / tradings_stocks_all['rate']
tradings_stocks_all['Price'] = tradings_stocks_all['Clean price']
tradings_stocks_all['FX Rate'] = tradings_stocks_all['trade rate'].combine_first(tradings_stocks_all['rate'])

In [173]:
cbonds_stocks_idx = tradings_stocks_all[~tradings_stocks_all["Price"].isna()].index
tradings_stocks_all.loc[cbonds_stocks_idx, "Type"] = 'Акция'
tradings_stocks_all.loc[cbonds_stocks_idx, 'Source'] = 'Cbonds'
tradings_stocks_all.loc[cbonds_stocks_idx, 'ISIN in DB'] = 1

In [174]:
Prices.update(tradings_stocks_all.groupby("ISIN").first())

In [175]:
ResultPrices = pd.concat([ResultPrices, Prices[~Prices["Price"].isna()]], axis=0)
Prices = Prices[Prices['Price'].isna()]

In [176]:
Prices.shape

(2222, 16)

#### Шаг 3. MOEX

##### Шаг 3.0. RuData MOEX stage

In [61]:
HistoryStockBonds = RuDataDF("HistoryStockBonds").df
HistoryStockBonds.head()

HistoryStockBonds started
HistoryStockBonds finished. HistoryStockBonds shape (117304, 66)


,boardname,regnumber,isin,matdate,listname,facevalue,currencyid,volume,value,numtrades,prev,prevlegalcloseprice,openperiod,legalopenprice,legalcloseprice,closeperiod,waprice,trendclose,trendclspr,trendwap,trendwappr,openval,closeval,highbid,lowoffer,bid,offer,yieldatwap,yieldclose,accint,duration,marketprice,marketprice2,admittedquote,issuesize,mpvaltrd,mp2valtrd,admittedvalue,marketprice3,marketprice3tradesvalue,type,lastprice,iricpiclose,beiclose,couponpercent,couponvalue,buybackdate,lasttradedate,cbrclose,yieldtooffer,yieldlastcoupon,offerdate,faceunit,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,Т+: Облигации - безадрес.,12978082V,RU000A0ZZVE6,2025-12-04T00:00:00,1,100000.0,SUR,1,10077239.0,1,92.6,92.6,NaN,None,95.0,None,95.0,2.4,2.59,2.4,2.59,10077239.0,10077239.0,95.0000,95.0,93.0010,95.0,8.15,8.15,294970.3324,363.0,None,NaN,None,17500,0.0,0.0,None,92.1615,1.263493e+08,ofz_bond,95.0,NaN,NaN,2.88,2875.0,None,2024-11-22T00:00:00,None,NaN,NaN,None,EUR,stock,bonds,TQCB,4,95.0,95.0,95.0,95.0,RU000A0ZZVE6,RUS-25 EUR,2024-11-22T00:00:00,3,NaN
1,Т+: Облигации (EUR) - безадрес.,12978082V,RU000A0ZZVE6,2025-12-04T00:00:00,1,100000.0,EUR,0,0.0,0,97.0,92.6,NaN,None,95.0,None,95.0,NaN,NaN,2.4,2.59,0.0,0.0,NaN,NaN,NaN,NaN,8.15,NaN,2780.7400,363.0,None,NaN,None,17500,0.0,0.0,None,92.1615,1.263493e+08,ofz_bond,NaN,NaN,NaN,2.88,2875.0,None,2024-06-05T00:00:00,None,NaN,NaN,None,EUR,stock,bonds,TQOE,4,NaN,NaN,NaN,NaN,RU000A0ZZVE6,RUS-25 EUR,2024-11-22T00:00:00,3,NaN
2,Т+: Облигации - безадрес.,12978082V,RU000A0ZZVE6,2025-12-04T00:00:00,1,100000.0,SUR,0,0.0,0,95.0,95.0,NaN,None,95.0,None,NaN,NaN,NaN,NaN,NaN,0.0,0.0,93.0001,98.5,90.0001,98.5,NaN,NaN,301252.4884,362.0,None,NaN,None,17500,0.0,0.0,None,92.1615,1.263493e+08,ofz_bond,NaN,NaN,NaN,2.88,2875.0,None,2024-11-22T00:00:00,None,NaN,NaN,None,EUR,stock,bonds,TQCB,4,NaN,NaN,NaN,NaN,RU000A0ZZVE6,RUS-25 EUR,2024-11-25T00:00:00,3,NaN
3,Т+: Облигации (EUR) - безадрес.,12978082V,RU000A0ZZVE6,2025-12-04T00:00:00,1,100000.0,EUR,0,0.0,0,97.0,95.0,NaN,None,95.0,None,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2804.3000,362.0,None,NaN,None,17500,0.0,0.0,None,92.1615,1.263493e+08,ofz_bond,NaN,NaN,NaN,2.88,2875.0,None,2024-06-05T00:00:00,None,NaN,NaN,None,EUR,stock,bonds,TQOE,4,NaN,NaN,NaN,NaN,RU000A0ZZVE6,RUS-25 EUR,2024-11-25T00:00:00,3,NaN
4,Т+: Облигации - безадрес.,12978082V,RU000A0ZZVE6,2025-12-04T00:00:00,1,100000.0,SUR,0,0.0,0,95.0,95.0,NaN,None,95.0,None,NaN,NaN,NaN,NaN,NaN,0.0,0.0,93.0001,98.5,93.0000,98.5,NaN,NaN,306161.2653,361.0,None,NaN,None,17500,0.0,0.0,None,92.1615,1.263493e+08,ofz_bond,NaN,NaN,NaN,2.88,2875.0,None,2024-11-22T00:00:00,None,NaN,NaN,None,EUR,stock,bonds,TQCB,4,NaN,NaN,NaN,NaN,RU000A0ZZVE6,RUS-25 EUR,2024-11-26T00:00:00,3,NaN


In [62]:
HistoryStockShares = RuDataDF("HistoryStockShares").df
HistoryStockShares.head()

HistoryStockShares started
HistoryStockShares finished. HistoryStockShares shape (26869, 54)


,boardname,regnumber,isin,listname,facevalue,currencyid,prevlegalcloseprice,prev,legalopenprice,openperiod,closeperiod,openval,closeval,legalcloseprice,trendclose,trendclspr,highbid,bid,offer,lowoffer,waprice,trendwap,trendwappr,volume,marketprice,marketprice2,mp2valtrd,mpvaltrd,value,numtrades,issuesize,admittedquote,admittedvalue,monthlycapitalization,dailycapitalization,marketprice3,marketprice3tradesvalue,type,closeauctionprice,waval,lastprice,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,Т+: Акции и ДР - безадрес.,1-01-00085-A,RU0008958863,1,1.0,SUR,2.0300,2.0895,None,2.0970,None,2097.0,10455.0,2.0585,0.0015,0.07,2.2800,2.0910,2.0915,1.9970,2.0645,-0.0415,-1.97,28012000,None,2.0615,37430587.0,0.0,57830187.5,3512,39749359700,None,None,8.683248e+10,8.194331e+10,2.0615,37430587.0,common_share,2.06,0.0,2.0910,stock,shares,TQBR,4,2.0990,2.0200,2.0970,2.0910,MSNG,+МосЭнерго,2024-11-27T00:00:00,3,NaN
1,Т+: Акции и ДР - безадрес.,1-01-00085-A,RU0008958863,1,1.0,SUR,2.0585,2.0910,None,2.1190,None,21190.0,4175.0,2.0630,-0.0035,-0.17,2.2800,2.0845,2.0875,1.9565,2.0630,-0.0015,-0.07,30463000,None,2.0590,49976604.0,0.0,62838468.5,5286,39749359700,None,None,8.683248e+10,8.184393e+10,2.0590,49976604.0,common_share,2.06,0.0,2.0875,stock,shares,TQBR,4,2.1190,2.0250,2.1190,2.0875,MSNG,+МосЭнерго,2024-11-28T00:00:00,3,NaN
2,Т+: Акции и ДР - безадрес.,1-01-00085-A,RU0008958863,1,1.0,SUR,2.0630,2.0875,None,2.0850,None,2085.0,8410.0,2.1390,0.0150,0.72,2.2045,2.1015,2.1035,1.7995,2.1305,0.0675,3.27,27319000,None,2.1315,55033210.0,0.0,58198557.5,2852,39749359700,None,None,8.683248e+10,8.472576e+10,2.1315,55033210.0,common_share,2.14,0.0,2.1025,stock,shares,TQBR,4,2.1755,2.0735,2.0850,2.1025,MSNG,+МосЭнерго,2024-11-29T00:00:00,3,NaN
3,Т+: Акции и ДР - безадрес.,1-01-00085-A,RU0008958863,1,1.0,SUR,NaN,2.2955,None,2.2650,None,2265.0,15732.5,NaN,-0.0480,-2.09,2.2840,2.2470,2.2475,2.2050,2.2565,-0.0145,-0.64,1675000,None,NaN,0.0,0.0,3779412.0,297,39749359700,None,None,9.752505e+10,9.058879e+10,NaN,0.0,common_share,NaN,0.0,2.2475,stock,shares,TQBR,4,2.2685,2.2405,2.2650,2.2475,MSNG,+МосЭнерго,2024-10-30T00:00:00,2,NaN
4,Т+: Акции и ДР - безадрес.,1-01-00085-A,RU0008958863,1,1.0,SUR,NaN,2.2475,None,2.1505,None,2150.5,2164.5,NaN,-0.0830,-3.69,2.5095,2.1635,2.1645,2.1350,2.1625,-0.1110,-4.88,2084000,None,NaN,0.0,0.0,4506233.5,398,39749359700,None,None,9.752505e+10,8.683248e+10,NaN,0.0,common_share,NaN,0.0,2.1645,stock,shares,TQBR,4,2.1745,2.1500,2.1505,2.1645,MSNG,+МосЭнерго,2024-10-31T00:00:00,2,NaN


In [63]:
HistoryStockNdm = RuDataDF("HistoryStockNdm").df
HistoryStockNdm.head()

HistoryStockNdm started
HistoryStockNdm finished. HistoryStockNdm shape (22818, 47)


,boardname,closeval,currencyid,facevalue,isin,issuesize,listname,matdate,numtrades,openval,prev,regnumber,trendclose,trendclspr,trendwap,trendwappr,type,value,volume,waprice,yieldatwap,yieldclose,lastprice,iricpiclose,beiclose,couponpercent,couponvalue,buybackdate,lasttradedate,cbrclose,yieldtooffer,yieldlastcoupon,offerdate,faceunit,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,РПС: Облигации (EUR) - адрес.,0.0,EUR,100000.0,RU000A1034K8,10000,1,2036-05-27T00:00:00,0,0.0,68.99,12978104V,NaN,NaN,NaN,NaN,ofz_bond,0.0,0,NaN,NaN,NaN,None,None,None,2.65,2650.0,None,2024-01-17T00:00:00,None,NaN,NaN,None,EUR,stock,ndm,PSEO,4,NaN,NaN,NaN,NaN,RU000A1034K8,RUS-36 EUR,2024-11-22T00:00:00,2,NaN
1,РПС: Облигации - адрес.,0.0,SUR,100000.0,RU000A1034K8,10000,1,2036-05-27T00:00:00,0,0.0,72.25,12978104V,NaN,NaN,NaN,NaN,ofz_bond,0.0,0,NaN,NaN,NaN,None,None,None,2.65,2650.0,None,2024-05-27T00:00:00,None,NaN,NaN,None,EUR,stock,ndm,PSOB,4,NaN,NaN,NaN,NaN,RU000A1034K8,RUS-36 EUR,2024-11-22T00:00:00,2,NaN
2,РПС с ЦК: Облигации - адрес.,0.0,SUR,100000.0,RU000A1034K8,10000,1,2036-05-27T00:00:00,0,0.0,52.00,12978104V,NaN,NaN,NaN,NaN,ofz_bond,0.0,0,NaN,NaN,NaN,None,None,None,2.65,2650.0,None,2024-09-09T00:00:00,None,NaN,NaN,None,EUR,stock,ndm,PTOB,4,NaN,NaN,NaN,NaN,RU000A1034K8,RUS-36 EUR,2024-11-22T00:00:00,2,NaN
3,РПС с ЦК: Облигации (EUR) - адрес.,0.0,EUR,100000.0,RU000A1034K8,10000,1,2036-05-27T00:00:00,0,0.0,91.50,12978104V,NaN,NaN,NaN,NaN,ofz_bond,0.0,0,NaN,NaN,NaN,None,None,None,2.65,2650.0,None,2022-08-25T00:00:00,None,NaN,NaN,None,EUR,stock,ndm,PTOE,4,NaN,NaN,NaN,NaN,RU000A1034K8,RUS-36 EUR,2024-11-22T00:00:00,2,NaN
4,РПС: Облигации (EUR) - адрес.,0.0,EUR,100000.0,RU000A1034K8,10000,1,2036-05-27T00:00:00,0,0.0,68.99,12978104V,NaN,NaN,NaN,NaN,ofz_bond,0.0,0,NaN,NaN,NaN,None,None,None,2.65,2650.0,None,2024-01-17T00:00:00,None,NaN,NaN,None,EUR,stock,ndm,PSEO,4,NaN,NaN,NaN,NaN,RU000A1034K8,RUS-36 EUR,2024-11-25T00:00:00,2,NaN


In [64]:
HistoryStockCcp = RuDataDF("HistoryStockCcp").df
HistoryStockCcp.head()

HistoryStockCcp started
HistoryStockCcp finished. HistoryStockCcp shape (34392, 54)


,accint,admittedquote,admittedvalue,bid,boardname,closeperiod,closeval,currencyid,duration,facevalue,highbid,isin,issuesize,legalcloseprice,legalopenprice,listname,lowoffer,marketprice,marketprice2,marketprice3,marketprice3tradesvalue,matdate,mp2valtrd,mpvaltrd,numtrades,offer,openperiod,openval,prev,prevlegalcloseprice,regnumber,trendclose,trendclspr,trendwap,trendwappr,type,value,volume,waprice,yieldatwap,yieldclose,engine,market,boardid,decimals,high,low,open,close,secid,shortname,tradedate,tradingsession,counter
0,3.58,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,1.077031e+09,SUR,NaN,1000.0,19.7,RU000A1025A7,409796724,None,None,1,19.50,20.22,None,None,0.0,2028-10-18T00:00:00,0.0,0.0,20,None,None,17873129.84,19.1,None,29015RMFS,0.40,None,0.4000,NaN,ofz_bond,3.567355e+10,38537452,19.50,None,None,stock,ccp,EQRP,4,19.70,19.50,19.7,19.50,SU29015RMFS3,ОФЗ 29015,2024-10-30T00:00:00,1,NaN
1,3.58,None,None,None,РЕПО с ЦК - адрес.,None,5.904070e+03,SUR,NaN,1000.0,NaN,RU000A1025A7,409796724,None,None,1,NaN,20.22,None,None,0.0,2028-10-18T00:00:00,0.0,0.0,64,None,None,4920.06,NaN,None,29015RMFS,NaN,None,NaN,NaN,ofz_bond,4.431012e+05,443,NaN,None,None,stock,ccp,PSRP,4,NaN,NaN,NaN,NaN,SU29015RMFS3,ОФЗ 29015,2024-10-30T00:00:00,1,NaN
2,4.09,None,None,None,РЕПО с ЦК 1 день - безадрес.,None,2.311786e+08,SUR,NaN,1000.0,19.5,RU000A1025A7,409796724,None,None,1,19.01,20.45,None,None,0.0,2028-10-18T00:00:00,0.0,0.0,26,None,None,17864561.36,19.5,None,29015RMFS,-0.49,None,-0.2500,NaN,ofz_bond,3.595239e+10,38879441,19.25,None,None,stock,ccp,EQRP,4,19.50,19.01,19.5,19.01,SU29015RMFS3,ОФЗ 29015,2024-10-31T00:00:00,1,NaN
3,4.09,None,None,None,Аукцион РЕПО с ЦК 1 неделя - безадрес.,None,0.000000e+00,SUR,NaN,1000.0,NaN,RU000A1025A7,409796724,None,None,1,NaN,20.45,None,None,0.0,2028-10-18T00:00:00,0.0,0.0,2,None,None,0.00,NaN,None,29015RMFS,NaN,None,1.9099,NaN,ofz_bond,9.955965e+09,10766528,20.05,None,None,stock,ccp,FROW,4,20.05,20.05,NaN,NaN,SU29015RMFS3,ОФЗ 29015,2024-10-31T00:00:00,1,NaN
4,4.09,None,None,None,РЕПО с ЦК - адрес.,None,5.896890e+03,SUR,NaN,1000.0,NaN,RU000A1025A7,409796724,None,None,1,NaN,20.45,None,None,0.0,2028-10-18T00:00:00,0.0,0.0,63,None,None,4914.07,NaN,None,29015RMFS,NaN,None,NaN,NaN,ofz_bond,4.328797e+05,435,NaN,None,None,stock,ccp,PSRP,4,NaN,NaN,NaN,NaN,SU29015RMFS3,ОФЗ 29015,2024-10-31T00:00:00,1,NaN


##### Шаг 3.1. Tranform

In [177]:
TransformHistoryStockBonds = HistoryStockBonds.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockShares = HistoryStockShares.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockNdm = HistoryStockNdm.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")
TransformHistoryStockCcp = HistoryStockCcp.groupby("isin")['close'].last().rename("Price").rename_axis("ISIN")

In [178]:
Prices.update(TransformHistoryStockBonds)
Prices.update(TransformHistoryStockShares)
Prices.update(TransformHistoryStockNdm)
Prices.update(TransformHistoryStockCcp)

In [179]:
moex_idx = Prices[~Prices["Price"].isna()].index
Prices.loc[moex_idx, 'Source'] = 'RU Data (MOEX)'
Prices.loc[moex_idx, 'ISIN in DB'] = 1

In [180]:
ResultPrices = pd.concat([ResultPrices, Prices.loc[moex_idx]], axis=0)
Prices = Prices[Prices['Price'].isna()]

In [181]:
Prices.shape

(2128, 16)

#### Шаг 4. Finish

In [182]:
FullPrices = pd.concat([ResultPrices, Prices], axis=0)

##### Шаг 4.1. Update currency

In [183]:
FullPrices = FullPrices\
.reset_index()\
.merge(CurrencyRate[['currency', 'rate']], how='left', left_on='Currency', right_on='currency')\
.set_index("ISIN")

In [184]:
rub_price_idx = FullPrices[FullPrices["RUB Price"].isna()].index
FullPrices.loc[rub_price_idx, "FX Rate"] = FullPrices.loc[rub_price_idx, "rate"].fillna(1)
FullPrices.loc[rub_price_idx, "RUB Price"] = FullPrices.loc[rub_price_idx, "Price"] * FullPrices.loc[rub_price_idx, "FX Rate"]

In [185]:
FullPrices.drop(columns=['currency', 'rate'], inplace=True)

##### Шаг 4.2. Write excel

In [186]:
FullPrices['export_timestamp'] = dt.now().strftime("%Y%m%d%H%M%S")

In [187]:
with pd.ExcelWriter(f'./data/Output/prices.xlsx', datetime_format='dd.mm.yyyy') as writer: 
    FullPrices[~FullPrices["Price"].isna()].to_excel(writer)

In [188]:
with pd.ExcelWriter(f'./data/Output/prices_full.xlsx', datetime_format='dd.mm.yyyy') as writer: 
    FullPrices.to_excel(writer)

In [189]:
from functions.send_email import main
main("prices.xlsx", "prices_full.xlsx")